# ETL Pipeline Preparation
Create the ETL pipeline.

### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [13]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
# load messages dataset
messages = pd.read_csv("messages.csv")
messages.head(3)

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   

                                            original   genre  
0  Un front froid se retrouve sur Cuba ce matin. ...  direct  
1                 Cyclone nan fini osinon li pa fini  direct  
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct

In [3]:
# load categories dataset
categories = pd.read_csv("categories.csv")
categories.head(3)

id                                         categories
0   2  related-1;request-0;offer-0;aid_related-0;medi...
1   7  related-1;request-0;offer-0;aid_related-1;medi...
2   8  related-1;request-0;offer-0;aid_related-0;medi...

### 2. Remove duplicates.

In [4]:
# Check duplicates in each data set
print ("""
Messages\n: Shape of Dataframe {}, \n: Number of Unique IDs: {} \n
Categories\n: Shape of Dataframe {} \n: Number of Unique IDs : {}
""".format(messages.shape, messages["id"].nunique(), categories.shape, categories["id"].nunique()))


Messages
: Shape of Dataframe (26248, 4), 
: Number of Unique IDs: 26180 

Categories
: Shape of Dataframe (26248, 2) 
: Number of Unique IDs : 26180



In [5]:
# Seems some records are duplicates, have a look at them
messages[messages["id"].isin(messages.groupby("id")["id"].count().sort_values(ascending = False).index[0:3])]

id                                            message  \
5557    6327  jel2  Acte 5: 29 2chr7 Lev11 Ecl9 Ecl4 jos5 ch...   
5558    6327  jel2  Acte 5: 29 2chr7 Lev11 Ecl9 Ecl4 jos5 ch...   
10101  11503  HELP THE EARTHQUAKE VICTIMS IN HAITI http tiny...   
10102  11503  HELP THE EARTHQUAKE VICTIMS IN HAITI http tiny...   
21358  24779  Shelter materials (thick polyesters) are being...   
21359  24779  Shelter materials (thick polyesters) are being...   
21360  24779  Shelter materials (thick polyesters) are being...   

                                                original   genre  
5557   jel2  Acte 5 : 29 2chr7 Lev11 Ecl9 Ecl4 jos5 c...  direct  
5558   jel2  Acte 5 : 29 2chr7 Lev11 Ecl9 Ecl4 jos5 c...  direct  
10101                                                NaN  social  
10102                                                NaN  social  
21358                                                NaN    news  
21359                                                NaN    news  
21360                                                NaN    news

In [6]:
categories[categories["id"].isin(categories.groupby("id")["id"].count().sort_values(ascending = False).index[0:3])]

id                                         categories
5557    6327  related-2;request-0;offer-0;aid_related-0;medi...
5558    6327  related-2;request-0;offer-0;aid_related-0;medi...
10101  11503  related-1;request-0;offer-0;aid_related-1;medi...
10102  11503  related-1;request-1;offer-0;aid_related-1;medi...
21358  24779  related-1;request-0;offer-0;aid_related-1;medi...
21359  24779  related-1;request-0;offer-0;aid_related-1;medi...
21360  24779  related-1;request-0;offer-0;aid_related-1;medi...

In [7]:
# Remove Duplicates (All equal value records) from both datasets
tablenames = ["messages", "categories"]

for k in range(2):
    table = [messages, categories][k]
    print ("""----------------------------\nWorking on table {}..., \nOriginal table has: records {}, unique ids {}
    """.format(tablenames[k].upper(), table.shape, table["id"].nunique()))
    
    table.drop_duplicates(keep = "first", inplace = True)

    # Check duplicates again in each data set
    print ("""After Removing Duplicates, table {} has: records {}, unique ids: {}
    """.format(tablenames[k].upper(), table.shape, table["id"].nunique()))
    
    if table.shape[0] != table["id"].nunique():
        print ("""\nsome ids in table {} have more than 1 distinct records\n: {}"""\
               .format(tablenames[k].upper(), 
                       table[table["id"].isin(table.groupby("id")["id"].count()\
                                              .sort_values(ascending = False).index[0:1])]))
        
        

----------------------------
Working on table MESSAGES..., 
Original table has: records (26248, 4), unique ids 26180
    
After Removing Duplicates, table MESSAGES has: records (26180, 4), unique ids: 26180
    
----------------------------
Working on table CATEGORIES..., 
Original table has: records (26248, 2), unique ids 26180
    
After Removing Duplicates, table CATEGORIES has: records (26216, 2), unique ids: 26180
    

some ids in table CATEGORIES have more than 1 distinct records
:           id                                         categories
22027  25512  related-1;request-0;offer-0;aid_related-1;medi...
22028  25512  related-1;request-0;offer-0;aid_related-1;medi...


### 3. Transform `categories`

__1. Split `categories` into separate category columns.__
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. 

__2. Convert category values to just numbers 0 or 1.__

In [8]:
# get category_colnames by 
# 1. taking one value from categories column
# 2. split it into 36 values
# 3. taking the first part (prior to '-') as category colnames
category_colnames = categories.head(1)["categories"].str.split(";", expand = True)\
.apply(lambda x: x.str.split("-", expand = True)[0], axis = 0).values[0]

In [9]:
# split categories column into 36 columns and assign it to new columns in category_colnames
categories[category_colnames] = categories["categories"].str.split(";", expand = True)\
.apply(lambda x: x.str.split("-", expand = True)[1]).astype(int)

In [10]:
# drop original categories column from table categories
categories.drop("categories", axis = 1, inplace = True)

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [11]:
# merge datasets
df = messages.merge(categories, how = "outer", on = "id")
df.head(3)

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   

   request  offer  aid_related  medical_help  medical_products      ...        \
0        0      0            0             0                 0      ...         
1        0      0            1             0                 0      ...         
2        0      0            0             0                 0      ...         

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0            0                     0                0       0      0     0   
1            0                     0                1       0      1     0   
2            0                     0                0       0      0     0   

   earthquake  cold  other_weather  direct_report  
0           0     0              0              0  
1           0     0              0              0  
2           0     0              0              0  

[3 rows x 40 columns]

### 7. Save the clean dataset into an sqlite database.
Do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library.

In [14]:
engine = create_engine('sqlite:///InsertDatabaseName.db')
df.to_sql('InsertTableName', engine, index=False)